## Import Libraries

In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy
import h5py

from scipy.linalg import hankel
import itertools
import pingouin as pg
import seaborn as sns



%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'


## PC-Algorithm function

In [ ]:
def pc_algorithm(X, var_names, lag_zero=True, include_lagged_target=False, include_lagged_source=False,
                 corr_method='spearman'):
    """
    Run the PC algorithm for causal discovery.

    Parameters:\n
    - X (numpy.ndarray): The data matrix with samples in rows and variables in columns.
    - var_names (list): List of variable names corresponding to the columns of X.
    - lag_zero (bool, optional): Whether to include lag-0 correlations. Default is True.
    - include_lagged_target (bool, optional): Whether to include lagged target variables in conditioning set. Default is False.
    - include_lagged_source (bool, optional): Whether to include lagged source variables in conditioning set. Default is False.
    - corr_method (str, optional): The method for computing correlations. Default is 'spearman'.

    Returns:\n
    - The resulting adjacency matrix of the PC algorithm, values represent strength of links.
    """
    #Hyper-parameters
    tau_max = 2
    a = 0.05

    #Size of input data
    v = X.shape[1]  #number of variables
    n = X.shape[0]  #length of time series

    #Pre-allocation of Adj-matrix
    linear_index = list(np.arange(0, v * (tau_max + 1)))
    Adj_matrix = pd.DataFrame(index=linear_index, columns=var_names)
    Adj_matrix[:] = np.inf

    #Outer loop to indentify target variable
    for i in range(v):

        target = X[:, i]  #time series of target variable

        # List of potnetial parents (All variables are considered, including lag=0)
        parents = np.arange(0, v * (tau_max + 1))

        if include_lagged_target:
            # Remove only target variable from the pool of potential parents
            parents = np.delete(parents, i)
        else:
            # Remove target variable and its lagged versions from the pool of potential parents
            parents = np.delete(parents, np.arange(i, len(parents), v))

        #Iterate over potential parents
        for j in range(len(parents)):

            idx = parents[j]  #linear index of parent

            #time series of the source
            #variable is given by the remainder idx%v
            #lag is given by the quotient idx//v
            source = hankel(X[:, idx % v])[:, idx // v]

            # Construct the pool of conditioning variables Z
            Z = np.copy(parents)
            if include_lagged_source:
                # Z = parents - source variable
                removed = parents[j]
            else:
                # Z = parents - source variable - lagged versions of source variable
                removed = np.arange(parents[j], v * (tau_max + 1), v)
            Z = Z[Z != removed]

            exit = 0

            #Inner loop increase the size of conditioning set iteratively from 0 to len(Z)
            for Nz in range(len(Z)):

                # All possible combinations of size Nz that can be conditioned on
                combinations = list(itertools.combinations(Z, Nz))

                for k in range(len(combinations)):

                    cond_set = np.ones([n, Nz])

                    for ii in range(Nz):
                        idx = combinations[k][ii]
                        cond_set[:, ii] = hankel(X[:, idx % v])[:, idx // v]

                    ### Run the conditional Independence test: I(target, sources, cond_set) ###

                    print(str(i) + ', ' + str(parents[j]) + '; ' + '[' + str(combinations[k]) + ']')

                    # Concatenate arrays by column in the order: target, sources, cond_set
                    data = np.concatenate((target.reshape(n, 1), source.reshape(n, 1), cond_set), axis=1)
                    data = pd.DataFrame(data, columns=list(range(data.shape[1])))

                    # Partial correlation
                    result = pg.partial_corr(data=data, x=0, y=1, covar=list(np.arange(2, data.shape[1]))
                                             , method=corr_method).round(3)

                    corr = result['r'][0]
                    p_value = result['p-val'][0]

                    if (p_value > a):
                        Adj_matrix.iloc[parents[j], i] = np.nan  #no causal link exists
                        exit = 1
                        break
                    elif (p_value <= a):
                        I_min = min(Adj_matrix.iloc[parents[j], i], corr)
                        Adj_matrix.iloc[parents[j], i] = I_min

                if exit == 1:
                    break

    return Adj_matrix
